In [1]:
import time
import numpy as np
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
RL_COLS = ['rl_15', 'rl_16', 'rl_17', 'rl_18', 'rl_19',
           'rl_20', 'rl_21', 'rl_22', 'rl_23', 'rl_24', 
           'rl_25', 'rl_26', 'rl_27', 'rl_28', 'rl_29', 
           'rl_30', 'rl_31']

RL_COLS_REVERSED = [col for col in reversed(RL_COLS)]

SELECT_COLS = RL_COLS + ['A', 'C', 'G', 'T']

WINDOW_LEN = 15
THRESHOLD_PER_M = 5.

In [89]:
!ls ../data/

final_ckpt_with_labels.csv        openml
final_ckpt_wo_labels.csv          positive_strand_reads.tsv
final_selected_df.csv             positive_strand_start_sites.tsv
final_selected_df_per_M.csv       positive_strand_with_features.csv
keep_data.csv


In [3]:
df_start_sites = pd.read_csv("../data/positive_strand_start_sites.tsv", sep="\t")
df_reads = pd.read_csv("../data/positive_strand_reads.tsv", sep="\t")

# df_start_sites = pd.read_csv("../data/positive_strand_start_sites.tsv", sep="\t")
# df_reads = pd.read_csv("../data/positive_strand_reads.tsv", sep="\t")

df_start_sites.shape, df_reads.shape

((2109, 2), (4641652, 19))

In [5]:
def non_zero_read_lengths(row):
    """ Are all columns rl_15...rl_31 equal to 0 in this sample? """
    return any([row[col] for col in RL_COLS])

def calc_num_reads_at_pos(row):
    """ Sum of columns rl_15...rl_31 for a sample. """
    return sum([row[col] for col in RL_COLS])

def largest_read_length(row):
    """ Represents the read length of the longest read found in this sample. """
    largest_rl = 0
    for col in RL_COLS_REVERSED:
        if row[col] > 0:
            largest_rl = int(col.split('_')[1])
            break
            
    return largest_rl

df_reads['is_non_zero_length'] = df_reads.progress_apply(non_zero_read_lengths, axis=1)
df_reads['num_reads'] = df_reads.progress_apply(calc_num_reads_at_pos, axis=1)
df_reads['largest_rl'] = df_reads.progress_apply(largest_read_length, axis=1)

# Reads per million
df_reads['reads_per_M'] = df_reads['num_reads']*1000000/df_reads['num_reads'].sum()

100%|██████████| 4641652/4641652 [02:46<00:00, 27939.83it/s]


In [10]:
class DataSelector:
    
    def __init__(self, df, threshold=5.):
        self.df = df.copy()
        self.threshold = threshold
        
        self.df["is_selected"] = False
    
    def select(self, row):
        if not row['is_non_zero_length'] or row['reads_per_M'] < self.threshold:
            return
        
        largest_read_length = row['largest_rl']
        
        first_index = row['position'] - largest_read_length - 1
        last_index = row['position'] - 1
        
        self.df.loc[first_index:last_index, 'is_selected'] = True

In [11]:
# test_df = df_reads.loc[0:100000]
# print(test_df['is_non_zero_length'].value_counts())

# data_selector = DataSelector(test_df)
# _ = test_df.progress_apply(data_selector.select, axis=1)

# data_selector.df['is_selected'].value_counts()

# select_df = data_selector.df

# rs_row = select_df[select_df['is_non_zero_length']].sample(n=1)
# rs_position = rs_row.position

# largest_read_length = 0
# for col in RL_COLS_REVERSED:
#     if rs_row[col].values[0] > 0:
#         largest_read_length = int(col.split('_')[1])
#         break

# largest_read_length

# select_df = select_df.join(pd.get_dummies(select_df['letter']))

# def select_df_sample(df, idx, window_len, select_cols):
#     return df.loc[idx - window_len: idx + window_len][select_cols]

Divide by total number of reads and multiply by 1M. Threshold = 5 reads per M

Apply on full dataset

In [23]:
data_selector = DataSelector(df_reads, threshold=THRESHOLD_PER_M)
_ = df_reads.progress_apply(data_selector.select, axis=1)

100%|██████████| 4641652/4641652 [00:44<00:00, 105453.51it/s]


In [24]:
df_select_full = data_selector.df
df_select_full['is_selected'].value_counts()

In [27]:
df_select_full.to_csv("../data/final_selected_df_per_M.csv")

In [28]:
def one_hot_encode(df, col, remove_og_col=False):
    one_hot = pd.get_dummies(df[col])
    df = df.join(one_hot)
    
    if remove_og_col:
        df.drop([col], axis=1, inplace=True)
    
    return df

df_select_full = one_hot_encode(df_select_full, 'letter')

In [33]:
def select_and_flatten(df, idx, window_len, select_cols):
    selected = df.loc[idx - window_len: idx + window_len][select_cols].values
    flattened = selected.reshape(-1)
    
    return flattened

selected_indices = df_select_full[df_select_full['is_selected']].index
selected_data = np.array([select_and_flatten(df_select_full, idx, WINDOW_LEN, SELECT_COLS) for idx in selected_indices])

In [83]:
def generate_colnames(window_len, select_cols):
    """ Generates the names of the columns based on window length """

    locs = np.arange(-window_len, window_len + 1, 1)

    colnames = []
    for loc in locs:
        for col in select_cols:
            loc_str = str(loc) if loc < 0 else ('+' + str(loc))

            colnames.append(col + loc_str)
            
    return colnames
        
colnames = generate_colnames(WINDOW_LEN, SELECT_COLS)
print(len(colnames))

651


In [38]:
selected_data = pd.DataFrame(selected_data, columns=colnames)
selected_data['position'] = selected_indices + 1

In [64]:
df_merged = pd.merge(selected_data, df_start_sites, how='left', left_on='position', right_on='start_position')
df_merged['start_position'] = df_merged['start_position'].apply(lambda x: pd.notnull(x))
df_merged.rename({'start_position': 'is_start_position'}, axis=1, inplace=True)

In [72]:
df_merged['is_start_position'].value_counts()

False    113965
True       1348
Name: start_position, dtype: int64

In [79]:
df_merged['is_start_position'].value_counts(normalize=True)

False    0.98831
True     0.01169
Name: is_start_position, dtype: float64

In [75]:
df_merged.to_csv("../data/final_ckpt_with_labels.csv")

In [84]:
# selected_data.to_csv("../data/final_ckpt_wo_labels.csv")